In [58]:
import numpy as np
import scipy.stats as stats

In [59]:
#Problem 1
# Set the seed for reproducibility
np.random.seed(0)

# Sample size
n = 100000

# Generate samples for U and its antithetic variates
U = np.random.uniform(1, 2, n)
U_antithetic = 3 - U  # Antithetic variates for U

# Generate samples for Z and its antithetic variates
Z = np.random.normal(-1, np.sqrt(2), n)
Z_antithetic = - Z - 2  # Adjusted antithetic variates for Z

# Combine original and antithetic samples
combined_Z = (np.exp(Z) + np.exp(Z_antithetic))/2
combined_U = (np.exp(U) + np.exp(U_antithetic))/2
# Calculate e^(U+Z)
exp_values = combined_U * combined_Z

# Estimate the mean
theta_estimate = np.mean(exp_values)

# Calculate the 95% confidence interval
std_error = np.std(exp_values) / np.sqrt(2 * n)
confidence_interval = stats.norm.interval(0.95, loc=theta_estimate, scale=std_error)

print("estimated mean: ",theta_estimate, " confidence interval: ", confidence_interval)


estimated mean:  4.6540815622672875  confidence interval:  (4.620746613171156, 4.687416511363419)


In [60]:
#Problem 2
#(a)
Ua = np.random.uniform(-1,1, 10000) 
Ub = np.random.uniform(-1,1, 10000)
print(np.mean(2*Ua**2 + 2*Ub**2 < 3))

0.9649


Given:
$$
P(2U_a^2 + 2U_b^2 < 3 | U_a) = P\left(-\sqrt{\frac{3}{2}} - U_a^2 < U_b < \sqrt{\frac{3}{2}} - U_a^2 | U_a\right) 
$$
When it satisfies $$U_a^2 > \frac{1}{2}$$, we have:
$$P\left(-\sqrt{\frac{3}{2}} - U_a^2 < U_b < \sqrt{\frac{3}{2}} - U_a^2 | U_a\right) = \sqrt{\frac{3}{2}} - U_a^2 $$

Otherwise:
$$P\left(-\sqrt{\frac{3}{2}} - U_a^2 < U_b < \sqrt{\frac{3}{2}} - U_a^2 | U_a\right) = 1 $$

Therefore:
$$ P(2U_a^2 + 2U_b^2 < 3 | U_a) = \min\left\{\sqrt{\frac{3}{2}} - U_a^2, 1\right\} $$

We need to generate \(U_a\) and estimate:
$$ E\left[\min\left\{\sqrt{\frac{3}{2}} - U_a^2, 1\right\}\right] $$


In [61]:
#(b)
Ua = np.random.uniform(-1,1, 10000)
Pa = np.minimum(1, np.sqrt(3/2 - Ua**2)) 
print(np.mean(Pa))

0.9625776733534352


In [108]:
#problem3
import numpy as np
from scipy.stats import norm

# Given parameters
S = 100  # initial stock price
K = [98, 100, 102]  # strike prices
r = 0.01  # risk-free rate
sigma = 0.3  # volatility
T = 1  # time to maturity
m = 4  # number of observation points
delta = 0.05  # confidence interval
lenK = len(K)  # number of strike prices

# Function to calculate prices and half-widths without control variates
def price_without_control_variates(N):
    M = np.tril(np.ones((m, m)), 0)  # lower triangular matrix
    Z = np.random.randn(m, N)
    V = np.dot(M, Z)
    A_T = np.dot(S*np.ones((1, m)), np.exp((r-sigma**2/2)*T/m*(np.arange(1, m+1).reshape(1, m).T*np.ones((1, N))) +
                 sigma*np.sqrt(T/m)*V)) / m

    p = np.zeros(lenK)
    D = np.zeros(lenK)
    for k in np.arange(lenK):
        X = np.exp(-r*T)*np.maximum(A_T-K[k], 0)  # discounted Asian call payoff
        p[k] = np.mean(X)
        D[k] = norm.ppf(1-(1-delta)/2)*np.std(X)/np.sqrt(N)
    
    return p, D

# Function to calculate prices and half-widths with control variates
def price_with_control_variates(N):
    M = np.tril(np.ones((m, m)), 0)
    Z = np.random.randn(m, N)
    V = np.dot(M, Z)

    # Control variate A_T
    A_T = np.dot(S*np.ones((1, m)), np.exp((r-sigma**2/2)*T/m*(np.arange(1, m+1).reshape(1, m).T*np.ones((1, N))) +
                 sigma*np.sqrt(T/m)*V)) / m
    A_Tmean = np.dot(S*np.ones((1, m)), np.exp(r*np.arange(1, m+1).reshape(1, m).T*T/m)) / m
    cstar = np.zeros((lenK, 1))

    for k in np.arange(lenK):
        x_var = np.hstack([np.ones((1, N)).T, (A_T - A_Tmean).T])
        output = np.linalg.lstsq(x_var, np.exp(-r*T)*np.maximum(A_T-K[k], 0).T, rcond=None)
        cstar[k] = -output[0][1]  # the second beta of the slopes

    # Recalculate A_T for new Z
    Z = np.random.randn(m, N)
    V = np.dot(M, Z)
    A_T = np.dot(S*np.ones((1, m)), np.exp((r-sigma**2/2)*T/m*(np.arange(1, m+1).reshape(1, m).T*np.ones((1, N))) +
                 sigma*np.sqrt(T/m)*V)) / m
    p_c = np.zeros(lenK)
    D_c = np.zeros(lenK)

    for k in np.arange(lenK):
        Cont = A_T - K[k]
        Y = np.exp(-r*T)*np.maximum(Cont, 0) + cstar[k]*((Cont + K[k]) - A_Tmean)
        p_c[k] = np.mean(Y)  # prices for 3 diff strikes
        D_c[k] = norm.ppf(1-(1-delta)/2)*np.std(Y)/np.sqrt(N)  # half-width

    return p_c, D_c

# Calculate prices and half-widths without control variates
prices_without_cv, half_widths_without_cv = price_without_control_variates(10000)

# Calculate prices and half-widths with control variates
prices_with_cv, half_widths_with_cv = price_with_control_variates(10000)

# results
print('Results without control variates (price and half-widths):')
print('Prices:', prices_without_cv)
print('Half-widths:', half_widths_without_cv)

print('\nResults with control variates (price and half-widths):')
print('Prices:', prices_with_cv)
print('Half-widths:', half_widths_with_cv)


Results without control variates (price and half-widths):
Prices: [9.43140237 8.45300491 7.55237638]
Half-widths: [0.00903328 0.00863518 0.008233  ]

Results with control variates (price and half-widths):
Prices: [9.44384549 8.46705652 7.56607339]
Half-widths: [0.00374279 0.00384234 0.00391508]


By including $A_T$ as an additional control variate, we observe an improvement in the half-lengths of the confidence intervals, which have decreased, indicating a more precise estimate. The reductions in half-lengths are approximately $60\%$, $57\%$, and $54\%$ for the respective strike prices, which shows that using two control variates can result in a more accurate estimation of the option price with tighter confidence intervals.


In [106]:
#Problem 4
 
sample = 100000
#rates
lam_x = 1/2
lam_y = 1/3

U = np.random.uniform(0,1,sample)

#Inverse Transform for exponential
Y = -3*np.log(U)

#Conditional expecation
P = 1-np.exp(-lam_x*(4-Y))
P[Y>=4]=1
print('probability: ', np.mean(P))

probability:  0.7432757026760688


In [107]:
#Problem 5


# Parameters
sample_size = 100000
lambda_1, lambda_2 = 1, 2  
lambda_1_prime, lambda_2_prime = 0.5, 1  

# Generate samples from importance distributions
samples_X1 = np.random.exponential(1/lambda_1_prime, sample_size)
samples_X2 = np.random.exponential(1/lambda_2_prime, sample_size)

# Calculate likelihood ratios for each sample
L_X1 = (lambda_1 / lambda_1_prime) * np.exp((lambda_1_prime - lambda_1) * samples_X1)
L_X2 = (lambda_2 / lambda_2_prime) * np.exp((lambda_2_prime - lambda_2) * samples_X2)

# Estimate probability
V = np.maximum(samples_X1, samples_X2)
event_indicator = (V > 20).astype(int)
probability_estimate = np.mean(event_indicator * L_X1 * L_X2)

print(f"Estimated probability P(V > 20): {probability_estimate}")


Estimated probability P(V > 20): 2.495160085837659e-10
